In [1]:
import os
import face_recognition as fr
import os, cv2
import numpy as np
import pandas as pd
from datetime import datetime as dt

/home/devjith/Luminar/Deep Learning/venv/lib/python3.12/site-packages/face_recognition_models/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
path = '/home/devjith/Luminar/Deep Learning/files/Face_images_celeb'

In [3]:
face_list = os.listdir(path)
face_list

['kohli.jpeg',
 'ronaldo.jpeg',
 'Devjith.jpg',
 'dhoni.jpeg',
 'alin_jose.jpeg',
 'mammooty.jpeg']

In [4]:
imgs = []
classnames = []

for i in face_list:
    i_path = os.path.join(path,i)
    img = cv2.imread(i_path)
    imgs.append(img)
    classnames.append(i.split('.')[0])


In [5]:
classnames

['kohli', 'ronaldo', 'Devjith', 'dhoni', 'alin_jose', 'mammooty']

In [6]:
def face_encode(imgs):
    encoding = []
    for i in imgs:
        img = cv2.cvtColor(i,cv2.COLOR_BGR2RGB)
        face_in_frame = fr.face_locations(img)
        encoding.append(fr.face_encodings(img,face_in_frame)[0])
    return encoding

In [7]:
encoded_faces_known = face_encode(imgs)
attendence_dict = {}

In [32]:
import pyttsx3

text = pyttsx3.init()

voice = text.getProperty('voices')

text.setProperty('rate',150)
text.setProperty('volume',0.2)

text.setProperty('voice',voice[1].id)



In [33]:
video = cv2.VideoCapture(0)
time = dt.now().strftime("%Y-%m-%d %H:%M:%S")
attendence = []
flag = False
counter = 0
while True:
    succ,raw = video.read()
    img = cv2.cvtColor(raw,cv2.COLOR_BGR2RGB)

    face_in_frame = fr.face_locations(img)
    encoded = fr.face_encodings(img,face_in_frame)

    for loc, enc in zip(face_in_frame,encoded):
        matches = fr.compare_faces(encoded_faces_known,enc)
        distance = fr.face_distance(encoded_faces_known,enc)
        index = np.argmin(distance)
        if matches[index]:
            name = classnames[index]
            y1,x2,y2,x1 = loc
            cv2.rectangle(raw,(x1,y1),(x2,y2),color=(0,0,255),thickness=2)
            cv2.putText(raw,f"{name.upper()}",(x1,y1-15),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
            attendence.append(name)
            counter+=1
            if counter > 50:
                
                text.say(f"Welcome {name}!")
                text.runAndWait()
                flag = True
        else:
            y1,x2,y2,x1 = loc
            cv2.rectangle(raw,(x1,y1),(x2,y2),color=(0,0,255),thickness=2)
            cv2.putText(raw,f"Unknown",(x1,y1-15),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
    cv2.imshow("Window",raw)
    if flag:
        break
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

video.release()
cv2.destroyAllWindows()




name = set(attendence)

for i in classnames:
    if i in name:
        attendence_dict[i] = ["Present",time]








df = pd.DataFrame(
    [(name, value[0], value[1]) for name, value in attendence_dict.items()],
    columns=["name", "attendance", "time"]
)
df.to_csv("Attendece.csv",index=False)

In [34]:
attendence_dict

{'Devjith': ['Present', '2025-12-10 22:01:15']}